In [1]:
import numpy as np
import pandas as pd
import os, sys
from scipy import signal

CURRENT_DIR = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(CURRENT_DIR+'/func'))
from func.photonic_func import Photonic

from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

nm = 1e-9
nsec = 1e-9
c = 2.99792458e8  # [m/s] Speed of light
hc = 1.987820871E-025  # [J * m / photon] Energy of photon with wavelength m

# Create WP with n shutters per cycle

In [2]:
photonic = Photonic(config='Cfg3')

freq = 45e6
shutter_per_cycle = 7
total_shutters = shutter_per_cycle + 6

shutters = photonic.generate_tof_shutters(op=photonic.op, freq=freq, 
                                  shutter_per_cycle=shutter_per_cycle, 
                                  total_shutters=total_shutters)                                                                           
shutters['dt'].shape, shutters['sh0'].shape, 

((100,), (100,))

In [3]:
data = [None] * total_shutters # Generate blank list with length equal number of shutters
for i in range(total_shutters):
    data[i] = go.Scatter(x=shutters['dt'], y=shutters['sh'+str(i)],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))

layout = dict(title='WP - separated',
              xaxis=dict(title='time delay [sec]',type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

In [4]:
data = [None] * shutter_per_cycle # Generate blank list with length equal number of shutters
for i in range(shutter_per_cycle):
    data[i] = go.Scatter(x=shutters['dt'], y=shutters['sh_comb'+str(i)],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))

layout = dict(title='WP - combined',
              xaxis=dict(title='time delay [sec]',type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))

# Calc phase of the phase modulation

In [5]:
# def get_modulation_phase(shutters, shutter_per_cycle, initial_phase = -3.0):
#     # Initialize phase components
#     XX = np.zeros(shutters['sh0'].shape)
#     YY = np.zeros(shutters['sh0'].shape)
    
#     # Extract phase components from the phasors
#     for i in range(shutter_per_cycle):
#         XX = shutters['sh_comb'+str(i)] * np.cos(2*np.pi*i/shutter_per_cycle + initial_phase) + XX
#         YY = shutters['sh_comb'+str(i)] * np.sin(2*np.pi*i/shutter_per_cycle + initial_phase) + YY

#         # Unwrap the phase
#     phase = np.arctan2(XX, YY)
#     return phase
phase = photonicsget_modulation_phase(shutters, shutter_per_cycle, initial_phase = -3.0)   

diff_phase = np.diff(phase) * 3.
diff_phase[np.abs(diff_phase) > 1.5] = np.nan

NameError: name 'photonicsget_modulation_phase' is not defined

In [ ]:
data = 2 * [None]
data[0] = go.Scatter(x=shutters['dt'], y=phase,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='phase')

data[1] = go.Scatter(x=shutters['dt'][:-1], y=diff_phase,
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='phase derivative')

layout = dict(title='WP - phase',
              xaxis=dict(title='time delay [sec]', type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal', type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))